In [1]:
import sys
sys.path = ["../needle_shape_sensing", *sys.path]

sys.path

['../needle_shape_sensing',
 '/home/dlezcan1/dev/git/python-needle-shape-sensing/notebooks',
 '',
 '/opt/ros/humble/lib/python3.10/site-packages',
 '/opt/ros/humble/local/lib/python3.10/dist-packages',
 '/home/dlezcan1/dev/git/python-needle-shape-sensing',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '/home/dlezcan1/.local/lib/python3.10/site-packages',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages']

In [2]:
%matplotlib notebook
import os
import numpy as np
import pandas as pd
import json

import matplotlib.pyplot as plt

import needle_shape_sensing as nss # current version of package

In [3]:
def plot_3d(shape):
    fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
    
    ax.plot(shape[:, 0], shape[:, 1], shape[:, 2])
    
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("z")
    
    return fig, ax

# plot_3d

def plot_3d_flat(shape):
    fig, ax = plt.subplots()
    
    for i, axis in enumerate(["x", "y"]):
        ax.plot(shape[:, 2], shape[:, i], '*-', label=axis)
        
    ax.set_xlabel("z")
    ax.set_ylabel("x/y")
    ax.legend()
    
    return fig, ax

# plot_3d_flat

def axisEqual3D( ax ):
    """ taken from online """
    extents = np.array( [ getattr( ax, 'get_{}lim'.format( dim ) )() for dim in 'xyz' ] )
    sz = extents[ :, 1 ] - extents[ :, 0 ]
    centers = np.mean( extents, axis=1 )
    maxsize = max( abs( sz ) )
    r = maxsize / 2
    for ctr, dim in zip( centers, 'xyz' ):
        getattr( ax, 'set_{}lim'.format( dim ) )( ctr - r, ctr + r )


# axisEqual3D

In [4]:
from shape_sensing import ShapeSensingFBGNeedle
import numerical
import intrinsics

In [5]:
fbgparam_file = os.path.join("../data", "needle_params_2022-10-10_Jig-Calibration_all_weights.json")

ss_fbgneedle_old = nss.shape_sensing.ShapeSensingFBGNeedle.load_json(
    fbgparam_file
)

ss_fbgneedle = ShapeSensingFBGNeedle.load_json(
    fbgparam_file
)

print("old:")
print(ss_fbgneedle_old)
print(100*"=")
print("new:")
print(ss_fbgneedle)

old:
Serial Number: 2CH-4AA-0001
Needle length (mm): 165.0
Diameter (mm): 1.27
Bending Moment of Insertia (mm^4): 0.12769820203693033
Torsional Moment of Insertia (mm^4): 0.25539640407386066
Young's Modulus, Emod (N/mm^2): 200000
Torsional Young's Modulus, Gmod (N/mm^2): 77519.37984496124
Number of FBG Channels: 2
Number of Active Areas: 4
Sensor Locations (mm):
	1: 65.0
	2: 100.0
	3: 135.0
	4: 155.0
Calibration Matrices:
	65.0: [[-1.389232062371995, 4.788991980135245], [-5.011010120953675, 1.173264147999622]] | weight: 0.396476763225053
	100.0: [[-0.659999818936315, 3.242868912009538], [-2.724487365712445, 1.072489958135295]] | weight: 0.377702305364081
	135.0: [[-3.746215358135378, 3.420561154478782], [-3.851729578776803, 1.101790184901246]] | weight: 0.097712380157397
	155.0: [[-4.603426609513273, 16.621729159502966], [-11.307496807746592, 2.209333032396484]] | weight: 0.12810855125347
new:
Serial Number: 2CH-4AA-0001
Needle length (mm): 165.0
Diameter (mm): 1.27
Bending Moment of I

In [6]:
ss_fbgneedle.insertion_point

array([0., 0., 0.])

In [7]:
Rmat = np.tile(np.eye(3), (5, 1,1))

In [8]:
x = np.arange(Rmat.shape[0]*3).reshape(Rmat.shape[0],3)
print(x)
x[np.newaxis, -1].shape

[[ 0  1  2]
 [ 3  4  5]
 [ 6  7  8]
 [ 9 10 11]
 [12 13 14]]


(1, 3)

In [9]:
R2 = np.diag(np.arange(1, 4))
print("R2 =\n" + str(R2))
print(100*"=")
print(R2[np.newaxis] @ Rmat)

R2 =
[[1 0 0]
 [0 2 0]
 [0 0 3]]
[[[1. 0. 0.]
  [0. 2. 0.]
  [0. 0. 3.]]

 [[1. 0. 0.]
  [0. 2. 0.]
  [0. 0. 3.]]

 [[1. 0. 0.]
  [0. 2. 0.]
  [0. 0. 3.]]

 [[1. 0. 0.]
  [0. 2. 0.]
  [0. 0. 3.]]

 [[1. 0. 0.]
  [0. 2. 0.]
  [0. 0. 3.]]]


In [10]:
insertion_depth = 125
current_curvatures = np.random.randn(2, ss_fbgneedle.num_activeAreas)

ss_fbgneedle.ref_wavelengths[:]     = 0
ss_fbgneedle_old.ref_wavelengths[:] = 0

ss_fbgneedle.current_curvatures     = current_curvatures
ss_fbgneedle_old.current_curvatures = ss_fbgneedle.current_curvatures

ss_fbgneedle.current_depth     = insertion_depth
ss_fbgneedle_old.current_depth = ss_fbgneedle.current_depth

In [11]:
ss_fbgneedle.insertion_point = np.array([0, 0, 0])

kc     = 0.002
w_init = np.array([kc, 0, 0])

pmat, Rmat     = ss_fbgneedle.get_needle_shape(kc, w_init)
pmat_o, Rmat_o = ss_fbgneedle_old.get_needle_shape(kc, w_init)

print(np.allclose(pmat, pmat_o), np.allclose(Rmat, Rmat_o))

True True


In [12]:
ss_fbgneedle.insertion_point = np.array([0, -10, 50])
ss_fbgneedle.current_depth = insertion_depth - ss_fbgneedle.insertion_point[2]

pmat, Rmat = ss_fbgneedle.get_needle_shape(kc, w_init)

mask_air = pmat[:, 2] < ss_fbgneedle.insertion_point[2]
pmat_air = pmat[mask_air]
Rmat_air = Rmat[mask_air]

Airdeflection.shape: (101, 3, 3)


In [ ]:
fig, ax = plot_3d(pmat)
ax.plot(
    ss_fbgneedle.insertion_point[0],
    ss_fbgneedle.insertion_point[1],
    ss_fbgneedle.insertion_point[2],
    'r*',
    label="insertion_point"
)
axisEqual3D(ax)

In [ ]:
fig, ax = plot_3d_flat(pmat)
ax.plot(
    ss_fbgneedle.insertion_point[2], ss_fbgneedle.insertion_point[1], 'ro'
)
ax.plot(
    ss_fbgneedle.insertion_point[2], ss_fbgneedle.insertion_point[0], 'ro'
)